In [1]:
%matplotlib inline
import mxnet as mx
import numpy as np

In [2]:
data = mx.sym.Variable('data')
label = mx.sym.Variable('softmax_label')

In [3]:
w1 = mx.sym.Variable('weight1')
b1 = mx.sym.Variable('bias1')
l1 = mx.sym.FullyConnected(data=data, num_hidden=128, name='layer1', weight=w1, bias=b1)
a1 = mx.sym.Activation(data=l1, act_type='relu', name='act1')

In [4]:
l2 = mx.sym.FullyConnected(data=a1, num_hidden=10, name='layer2')
cost_classification = mx.sym.SoftmaxOutput(data=l2, label=label)

In [5]:
batch_size = 128
input_shapes  ={'data':(batch_size, 28*28), 'softmax_label':(batch_size,)}
executor = cost_classification.simple_bind(
    ctx = mx.gpu(),
    grad_req = 'write',
    **input_shapes
)

In [6]:
executor_test = cost_classification.bind(
    ctx = mx.gpu(),
    grad_req = 'null',
    args = executor.arg_arrays
)

In [7]:
for r in executor.arg_arrays:
    r[:] = np.random.randn(*r.shape)*0.02

In [8]:
from skdata.mnist.view import OfficialVectorClassification

In [9]:
data = OfficialVectorClassification()
trIdx = data.sel_idxs[:]
teIdx = data.val_idxs[:]

In [10]:
print trIdx, teIdx
#np.random.shuffle(trIdx)
#print trIdx

[    0     1     2 ..., 59997 59998 59999] [50000 50001 50002 ..., 59997 59998 59999]


注意，下边训练过程中，除255一定要除255.，否则会导致一个问题，就是结果不是浮点数而是整数，最终拟合结果会非常差

In [13]:
for epoch in range(10):
    print  "Start epoch", epoch
    np.random.shuffle(trIdx)
    
    for x in range(0, len(trIdx), batch_size):
        batchX = data.all_vectors[trIdx[x:x+batch_size]]
        batchY = data.all_labels[trIdx[x:x+batch_size]]
        
        if batchX.shape[0] != batch_size:
            continue
        executor.arg_dict['data'][:] = batchX/255.
        executor.arg_dict['softmax_label'][:] = batchY
        executor.forward(is_train=True)
        executor.backward()
        
        for pname, W, G in zip(cost_classification.list_arguments(), executor.arg_arrays, executor.grad_arrays):
            if pname in ['data', 'softmax_label']:
                continue
            W[:] = W - G * .001 
        
    num_correct = 0
    num_total = 0
    for i in range(0, len(teIdx), batch_size):
        batchTX = data.all_vectors[teIdx[i:i+batch_size]]
        batchTY = data.all_labels[teIdx[i:i+batch_size]]

        if batchTX.shape[0] != batch_size:
            continue
        executor_test.arg_dict['data'][:] = batchTX/255.
        executor_test.forward()
        num_correct += sum(batchTY == np.argmax(executor_test.outputs[0].asnumpy(), axis=1))
        num_total += len(batchTY)
    print 'Accuracy thus far:', num_correct / float(num_total)

 Start epoch 0
Accuracy thus far: 0.949819711538
Start epoch 1
Accuracy thus far: 0.959635416667
Start epoch 2
Accuracy thus far: 0.963541666667
Start epoch 3
Accuracy thus far: 0.969651442308
Start epoch 4
Accuracy thus far: 0.972355769231
Start epoch 5
Accuracy thus far: 0.975060096154
Start epoch 6
Accuracy thus far: 0.977864583333
Start epoch 7
Accuracy thus far: 0.978265224359
Start epoch 8
Accuracy thus far: 0.981470352564
Start epoch 9
Accuracy thus far: 0.982772435897
